# **랜덤 포레스트**
랜덤 포레스트:랜덤서치를 활용해 학습시킨 여러대의 트리 모델을 활용하여 하나의 결과를 뽑아내는 앙상블구조

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
#sklearn.ensemble패키지 하위의 랜덤 포레스트 모델인 RandomForestClassifier클래스를 사용한다.
#이 클래스는 기본적으로 100개의 트리를 학습시킨다.

#RandomForestClassifier의 매개변수 n_jobs는 랜덤포래스트 모델이 여러개의 트리 모델을 학습시킬때 몇개의 코어를 사용할지 지정한다.
#n_jobs=-1일 경우 사용가능한 모든 코어를 사용하여 훈련시킨다.
rf = RandomForestClassifier(n_jobs=-1)
#return_train_score:True로 설정하면 cross_validate메서드의 반환결과 딕셔너리에 'train_score':(훈련세트 점수) 쌍을 추가해준다.
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9978352895600912 0.8910864736803139


In [3]:
rf.fit(train_input, train_target)
#feature_importances:특성 중요도
#[alchor sugar pH]
print(rf.feature_importances_)

[0.23138395 0.50619094 0.26242511]


In [4]:
#oob_score:True 랜덤 포레스트 모델의 oob(Out Of Bag)으로 검증한 점수를 도출한다.
#*oob(Out Of Bag):랜덤 포레스트 모델은 훈련세트에서 균등 분포 중복 샘플링(중복 조합)을 여러번 실행하여 여러개의 훈련세트로 분할하며,
#                 이 분할한 훈련세트들로 각각의 트리 모델을 학습시킨다. 이 때문에, 학습을 한 번 시킬때 마다 남는 요소들의 집합이 생기게 되며,
#                 n번 학습시킬 경우 n개의 이러한 집합들이 생기는데, 이를 합쳐서 oob(Out Of Bag)이라고 한다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8943621319992303


# **엑스트라 트리**
*엑스트라 트리:랜덤 포레스트 모델에서 부트스트랩 샘플을 사용하지 않고 노드분할을 무작위로 하여 성장을 규제하는 모델

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
#sklearn.ensemble패키지 하위에 기본적으로 포함되어있는 엑스트라 트리 모델인 ExtraTreeClassifier클래스를 사용한다.
#랜덤하게 노드를 분할하기 때문에 랜덤 포레스트 모델보다 속도가 빠르다.
#기본적으로 100개의 트리모델을 학습시킨다.

et = ExtraTreesClassifier(n_jobs=-1)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9978352895600912 0.8860840675205448


In [6]:
et.fit(train_input, train_target)
#[alchor sugar pH]
print(et.feature_importances_)

[0.18661911 0.52688588 0.28649501]


# **그레이디언트 부스팅**
*그레이디언트 부스팅:회귀모델일 경우 평균제곱오차 손실함수를, 분류모델일 경우 로지스트 손실함수를 활용하여, 손실함수의 함수값을 낮추는 방향의 트리를 추가하여 학습하는 모델

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
#sklearn.ensemble패키지 하위에 기본적으로 포함되어있는 그레이디언트 부스팅 모델인 GradientBoostingClassifier를 사용한다.
#그레이디언트 부스팅 모델은 손실함수가 낮아지는 방향으로 트리를 순차적으로 추가하기때문에 앞의 두 모델보다 학습속도가 느리다.

gb = GradientBoostingClassifier()
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8866169860396467 0.8703074331827942


In [8]:
#n_estimators:트리의 개수를 지정
#learning_rate:학습 비율(기본값:0.1). 커질수록 한번 학습시 모델에 반영하는 비율이 커진다.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9416968521370734 0.876077774487303


In [9]:
gb.fit(train_input, train_target)
#[alchor sugar pH]
print(gb.feature_importances_)

[0.16595299 0.67431535 0.15973166]


# **히스토그램 기반의 그레이디언트 부스팅**
*히스토그램 기반의 그레이디언트 부스팅:데이터를 몇 개의 구간으로 나누어 학습하는 그레이디언트 부스팅 알고리즘

In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
#sklearn.ensemble패키지 하위에 기본적으로 포함되어있는 히스토그램 기반의 그레이디언트 부스팅 모델인 HistGradientBoostingClassifier 사용한다.
#하지만 사이킥런에 추가된지 얼마 안됬으며, 실험적인 측면의 모델이기 때문에, 이를 사용할때는 반드시 sklearn.experimental패키지 하위에 포함되어있는
#enable_hist_gradient_boosting클래스를 임포트해줘야한다.

#HistGradientBoostingClassifier모델은 기본적으로 데이터를 256개의 구간으로 나누는데
#255개의 구간은 정상적인 데이터 값의 구간인데 비해 마지막 한구간은 누락된 데이터를 보관하는 구간이다.
#따라서 판다스 등으로 데이터를 다운받아 테스트할때, 누락된 값이 있어도 전처리 할 필요가 없다.

hgb = HistGradientBoostingClassifier()
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#HistGradientBoostingClassifier모델에는 특성중요도가 없다.

0.9325088519940576 0.8810799955578588


**permutation importance**

In [11]:
from sklearn.inspection import permutation_importance
#permutation importance(치환 중요도):각 특성값을 섞어서 성능이 가장 덜 하락한 특성 순서로 매긴 중요도.
#ex)[alchor,sugar,ph]인 데이터[[2,15,8],  인 가상의 데이터를 1.[[8,15,8], 2.[[2,150,8], 3.[[2,15,9],  으로 바꾸어
#                            [4,49,6],                   [2,49,6],    [4,15,6],     [4,49,8],
#                            [8,150,9]]                  [4,150,9]]   [8,49,9]]     [8,150,6]]
#원본 데이터와의 정확도를 비교하여 각 특성의 중요도를 검사한다.
#예를 들어, 원본 데이터의 정확도가 80%가 나오고 1번의 데이터(alchor값을 뒤바꾼 데이터)가 정확도 50%, 2번 데이터(sugar의 값을 뒤바꾼 데이터)가 정확도 70%가 나왔다면
#alchor보다 sugar가 더 중요한 값이 된다.

#permutation_importance는 분류,회귀 모델 상관없이 모든 모델에 적용이 가능한 클래스이다.

hgb.fit(train_input, train_target)
#n_repeats:각 특성당 몇번 섞어 결괏값을 낼건지 결정
#n_jobs:몇개의 코어를 사용할지 설정. -1일경우 가능한 모든 코어를 사용한다.
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, n_jobs=-1)
print(result.importances_mean)

[0.09328459 0.25039446 0.08262459]


In [12]:
#permutation_importance에 테스트세트를 넣고 검증할경우, 이 모델을 실전에 투입했을때 어떤 특성에 가중치가 붙는지 실험할 수 있다.
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, n_jobs=-1)
print(result.importances_mean)
#훈련세트를 넣었을 때와 마찬가지로 'sugar'특성의 중요도가 높음을 알 수 있다.

[0.054      0.19692308 0.04384615]


**XGBoost**

In [13]:
from xgboost import XGBClassifier

#tree_method='hist'라고 하면 XGBoost모델이 히스토그램기반으로 학습한다.
xgb = XGBClassifier(tree_method='hist')
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8800265736277282 0.8681896424076406


**LightGBM**

In [15]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9316910123260858 0.8816587695269119
